In [ ]:
from utils import harris, dist2, find_sift
from PIL import Image
from skimage.feature import plot_matches
from skimage.transform import ProjectiveTransform, warp
from scipy.linalg import svd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import numpy as np
# add your imports below

## 1. Load images and Convert to grayscale

In [ ]:
img1 = Image.open('hw2_data/uttower_left.JPG').convert('L')# 
img2 = Image.open('hw2_data/uttower_right.JPG').convert('L')#

img1_float = np.array(img1, dtype = np.float32) 
img2_float = np.array(img2, dtype = np.float32) 


In [ ]:
plt.imshow(img1, cmap='gray')

In [ ]:
plt.imshow(img2, cmap='gray')

## 2. Detect Feature Points

In [ ]:
sigma = 3.0
thresh = 50.0
radius = 3.0
# use harris from utils.py
cim1, r1, c1 = harris(img1_float, sigma, thresh, radius)#
cim2, r2, c2 = harris(img2_float, sigma, thresh, radius)#

In [ ]:
plt.imshow(cim1, cmap='gray')

In [ ]:
plt.imshow(cim2, cmap='gray')

In [ ]:
def draw_corners(img, r, c):
    img_copy = img.copy()
    fig, ax = plt.subplots(figsize = (10, 10))
    for i in range(0, len(r)):
        rect = patches.Rectangle((c[i], r[i]), 10, 10, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    ax.imshow(img_copy, cmap='gray')

In [ ]:
draw_corners(img1, r1, c1)

In [ ]:
draw_corners(img2, r2, c2)

## 3. Extract local neighborhoods around every keypoint

In [ ]:
def neighbor_descriptors(img, r, c, radius):
    descriptors = []
    size = 2 * radius + 1
    height, width = img.shape
    for y, x in zip(r, c):
        y_min = max(y - radius, 0)
        y_max = min(y + radius + 1, height)
        x_min = max(x - radius, 0)
        x_max = min(x + radius + 1, width)
        patch = img[y_min:y_max, x_min:x_max]
        y_top = max(radius - y, 0)
        y_bottom = max(y + radius + 1 - height, 0)
        x_left = max(radius - x, 0)
        x_right = max(x + radius + 1 - width, 0)
        patch = np.pad(patch, ((y_top, y_bottom), (x_left, x_right)), mode='constant')
        descriptor = patch.reshape(-1)
        descriptors.append(descriptor)
    return np.array(descriptors)

def normalize(mat):
    row_means = np.mean(mat, axis=1, keepdims=True)
    row_stds = np.std(mat, axis=1, keepdims=True)
    normalized_matrix = (mat - row_means) / (row_stds + 1e-8)

    return normalized_matrix

descriptors_1 = neighbor_descriptors(img1_float, r1, c1, radius=10)
descriptors_2 = neighbor_descriptors(img2_float, r2, c2, radius=10)

descriptors_1 = normalize(descriptors_1)
descriptors_2 = normalize(descriptors_2)
print(len(r1))
print(descriptors_1)
print(descriptors_1.shape)
print("\n")
print(len(r2))
print(descriptors_2.shape)
print(descriptors_2)

In [ ]:
def sift_descriptors(img, r, c, radius):
    r = r.reshape(-1, 1)
    c = c.reshape(-1, 1)  
    radius = np.full_like(r, radius)  
    circles = np.hstack([c, r, radius])  
    return find_sift(img, circles)


descriptors_1 = sift_descriptors(img1_float, r1, c1, 3)
descriptors_2 = sift_descriptors(img2_float, r2, c2, 3)
print(descriptors_1.shape)
print(descriptors_1)

print(descriptors_2.shape)
print(descriptors_2)


## 4. Compute distances between descriptors

In [ ]:
# use dist2 from utils.py to compute dist between descriptors
distances = dist2(descriptors_2, descriptors_1)#
print(distances)
print(distances.shape)

## 5. Select Matches

In [ ]:
def filter_descriptors_by_dist(distances, thresh):
    matches = []
    for i in range(distances.shape[0]):
        for j in range(distances.shape[1]):
            if distances[i][j] < thresh:
                matches.append((i, j))
    return np.array(matches)

print(np.max(distances))
thresh = np.sort(distances.flatten())[100]

print(thresh)

filtered_matches = filter_descriptors_by_dist(distances, thresh)

def plot_matches(img1, img2, r1, c1, r2, c2, matches):
    fig, ax = plt.subplots(nrows=1, ncols=1)
    img = np.concatenate((img1, img2), axis=1)
    ax.imshow(img, cmap='gray')
    for i, j in matches:
        ax.plot([c1[i], c2[j] + img1.shape[1]], [r1[i], r2[j]], 'r')
    # Show the plot
    plt.show()

print(len(filtered_matches))
plot_matches(img1_float, img2_float, r1, c1, r2, c2, filtered_matches)

## 6. RANSAC

In [ ]:
def RANSAC(filtered_matches, eps, num_loops):
    best_homography = None
    best_inliers = []
    best_residual = 0

    for _ in range(num_loops):
        sample_indices = np.random.choice(range(len(filtered_matches)), 4)
        
        src_points = np.array([[c1[i], r1[i]] for i, _ in filtered_matches[sample_indices]])
        dst_points = np.array([[c2[j], r2[j]] for _, j in filtered_matches[sample_indices]])


        homography = estimate_homography(src_points, dst_points)
        inliers, residual = count_inliers(filtered_matches, homography, eps)

        if len(inliers) > len(best_inliers):
            best_homography = homography
            best_inliers = inliers
            best_residual = residual

    return best_inliers, best_residual, best_homography

def estimate_homography(src_points, dst_points):
    num_points = src_points.shape[0]
    A = np.zeros((2 * num_points, 9))

    for i in range(num_points):
        x, y = src_points[i]
        u, v = dst_points[i]
        A[2 * i] = [-x, -y, -1, 0, 0, 0, x * u, y * u, u]
        A[2 * i + 1] = [0, 0, 0, -x, -y, -1, x * v, y * v, v]

    _, _, V = svd(A)
    H = V[-1].reshape((3, 3))

    return H

def count_inliers(matches, homography, eps):
    inliers = []
    residual = 0
    for i, j in matches:
        src_point = np.array([c1[i], r1[i], 1])
        dst_point = np.array([c2[j], r2[j]])
        transformed_src = homography.dot(src_point)
        
        if np.isclose(transformed_src[2], 0):
            continue

        transformed_src /= transformed_src[2]
        error = np.linalg.norm(transformed_src[:2] - dst_point)
        
        if error <= eps:
            inliers.append([i, j])
            residual += error

    return np.array(inliers), residual


In [ ]:
# report num of inliers and the average residual for the inliers
inliers, residual, H = RANSAC(filtered_matches, 2, 10000)
print("number of inliers: " + str(len(inliers)))
print("residual: " + str(residual))

# display the locations of inlier matches in both images
plot_matches(img1_float, img2_float, r1, c1, r2, c2, inliers)

## 7. Warp one image onto the other

In [ ]:
def warp_img(image_left, H):
    # this part of code is referenced from Ajinkya Tejankar
    # get the height and width of the image
    # this function will work with color images
    # so, having third color channel is not a problem
    h_left, w_left = image_left.shape[:2]

    # we want to find where the image corners are going to land
    # so, we create a matrix of four corner points
    C_left = np.array([
        [0, 0     , w_left, w_left],
        [0, h_left, 0     , h_left],
        [1, 1     , 1     , 1     ]
    ])

    # apply the homography to the corner points to get projected corner points
    Cp_left = H @ C_left
    Cp_left = Cp_left / Cp_left[-1, :]

    # find the minimum height and width of the projected corners
    w_min, h_min = Cp_left[:-1].min(axis=1).tolist()
    # we might need to properly floor or ceil the floats to prevent
    # the edge pixels from getting cropped but this works for our needs
    # feel free to fix this
    w_min, h_min = int(np.abs(w_min)), int(np.abs(h_min))
    # what's the final warped image size that can hold the full image?
    warped_image_shape = (h_left + h_min, w_left + w_min)

    # we create a new homography that applies the translation
    # that would be otherwise cropped by the warp function below
    Ht = np.array([
        [1, 0, w_min],
        [0, 1, h_min],
        [0, 0, 1    ]
    ])
    # apply the translation homography so that the image is warped
    # but does not have a negative translation relative to origin
    Hw = Ht @ H
    # may not be strictly necessary but make sure that (3,3) is 1
    Hw = Hw / Hw[-1, -1]

    # use skimage.transform.ProjectiveTransform to create a transform for the homography Hw
    tform = ProjectiveTransform(Hw)#
    # use skimage.transform.warp to apply the transform
    warped_image = warp(image_left, tform.inverse, output_shape=warped_image_shape)#

    return warped_image

In [ ]:
img1_np = np.array(img1)
print(img1_np.shape)
warped_img1 = warp_img(img1_np, H)
plt.imshow(warped_img1, cmap='gray')
print(warped_img1.shape)

## 8. Create a new image to hold the panorama

In [ ]:
def create_panorama(warped_img1, img2):
    img2 = np.array(img2)
    h1, w1 = warped_img1.shape
    h2, w2 = img2.shape
    print(h2, w2)
    offset_h, offset_w = h1-h2, w1-w2
    
    Ht = np.array([
        [1, 0, offset_w],
        [0, 1, offset_h],
        [0, 0, 1    ]
    ])
    warped_image_shape = (h2+offset_h, w2 + offset_w)
    print(warped_image_shape)

    tform = ProjectiveTransform(Ht)#
    # use skimage.transform.warp to apply the transform
    warped_image = warp(img2, tform.inverse, output_shape = warped_image_shape)#
    warped_image[:h1, :w1] += warped_img1
    return warped_image


# create a panorama in gray scale first
panorama_gray = create_panorama(warped_img1, img2)
plt.imshow(panorama_gray, cmap='gray')

In [ ]:
# then create a color image by doing the same for each channel

def create_panorama_rgb(img1, img2):
    img1 = np.array(img1)
    img2 = np.array(img2)

    panorama_r = create_panorama(img1[..., 0], img2[..., 0])
    panorama_g = create_panorama(img1[..., 1], img2[..., 1])
    panorama_b = create_panorama(img1[..., 2], img2[..., 2])

    # Stack all the channels together
    panorama = np.stack([panorama_r, panorama_g, panorama_b], axis=-1)

    return panorama

img1_color = img1_float
img2_color = img2_float
panorama = create_panorama_rgb(img1_color, img2_color)
plt.imshow(panorama)
